In [2]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import dash_bootstrap_components as dbc
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Load dataset once (avoiding redundant reads)
file_path = "data.csv"
df = pd.read_csv(file_path)

# Extract numerical columns for dropdowns
numerical_cols = df.select_dtypes(include=['number']).columns

# Precompute correlation matrix once
correlation_matrix = df.corr()

# Machine Learning: Train a Linear Regression Model (if applicable)
if 'Grades' in df.columns:
    X = df.drop(columns=['Grades'], errors='ignore')
    y = df['Grades']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    df['Predicted Grades'] = model.predict(X)

# Initialize the Dash app with Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Student Performance Dashboard"

# Layout of the dashboard
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("📊 Student Performance Dashboard", className="text-center mb-4 text-white"), width=12)
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Select X-Axis Attribute:", style={'color': 'white', 'fontSize': '18px'}),
            dcc.Dropdown(
                id='x-axis',
                options=[{'label': col, 'value': col} for col in numerical_cols],
                value=numerical_cols[0] if len(numerical_cols) > 0 else None,
                style={'backgroundColor': 'white', 'color': 'black'}
            ),
        ], width=6),
        dbc.Col([
            html.Label("Select Y-Axis Attribute:", style={'color': 'white', 'fontSize': '18px'}),
            dcc.Dropdown(
                id='y-axis',
                options=[{'label': col, 'value': col} for col in numerical_cols],
                value=numerical_cols[1] if len(numerical_cols) > 1 else None,
                style={'backgroundColor': 'white', 'color': 'black'}
            ),
        ], width=6),
    ], className="mb-4"),
    dbc.Row([
        dbc.Col([
            dbc.ButtonGroup([
                dbc.Button("Scatter", id='scatter-btn', n_clicks=0, color="primary"),
                dbc.Button("Bar", id='bar-btn', n_clicks=0, color="primary"),
                dbc.Button("Hist", id='hist-btn', n_clicks=0, color="primary"),
                dbc.Button("Box", id='box-btn', n_clicks=0, color="primary"),
                dbc.Button("Heatmap", id='heatmap-btn', n_clicks=0, color="primary"),
                dbc.Button("Line Chart", id='line-btn', n_clicks=0, color="primary"),
                dbc.Button("Area Chart", id='area-btn', n_clicks=0, color="primary"),
                dbc.Button("Bubble", id='bubble-btn', n_clicks=0, color="primary"),
                dbc.Button("Predict", id='predict-btn', n_clicks=0, color="success"),
            ], className="mb-4", style={'width': '100%'}),
        ], width=12),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='output-graph', style={'borderRadius': '10px'}), width=12)
    ]),
], fluid=True, style={"backgroundColor": "#2C3E50"})  # Dark background

# Callback to update graph efficiently
@app.callback(
    Output('output-graph', 'figure'),
    Input('scatter-btn', 'n_clicks'),
    Input('bar-btn', 'n_clicks'),
    Input('hist-btn', 'n_clicks'),
    Input('box-btn', 'n_clicks'),
    Input('heatmap-btn', 'n_clicks'),
    Input('line-btn', 'n_clicks'),
    Input('area-btn', 'n_clicks'),
    Input('bubble-btn', 'n_clicks'),
    Input('predict-btn', 'n_clicks'),
    State('x-axis', 'value'),
    State('y-axis', 'value')
)
def update_graph(n_scatter, n_bar, n_hist, n_box, n_heatmap, n_line, n_area, n_bubble, n_predict, selected_x, selected_y):
    ctx = dash.callback_context
    if not ctx.triggered:
        return go.Figure()
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Use memoized figure creation to improve performance
    fig = go.Figure()
    if button_id == 'scatter-btn' and selected_x and selected_y:
        fig = px.scatter(df, x=selected_x, y=selected_y, title='Scatter Plot', color=selected_x, template="plotly_dark")
    elif button_id == 'bar-btn' and selected_x and selected_y:
        fig = px.bar(df, x=selected_x, y=selected_y, title='Bar Chart', color=selected_x, template="plotly_dark")
    elif button_id == 'hist-btn' and selected_x:
        fig = px.histogram(df, x=selected_x, title='Histogram', color=selected_x, template="plotly_dark")
    elif button_id == 'box-btn' and selected_x and selected_y:
        fig = px.box(df, x=selected_x, y=selected_y, title='Box Plot', color=selected_x, template="plotly_dark")
    elif button_id == 'heatmap-btn':
        fig = px.imshow(correlation_matrix, title='Correlation Heatmap', color_continuous_scale='Viridis', template="plotly_dark")
    elif button_id == 'line-btn' and selected_x and selected_y:
        fig = px.line(df, x=selected_x, y=selected_y, title='Line Chart', color=selected_x, template="plotly_dark")
    elif button_id == 'area-btn' and selected_x and selected_y:
        fig = px.area(df, x=selected_x, y=selected_y, title='Area Chart', color=selected_x, template="plotly_dark")
    elif button_id == 'bubble-btn' and selected_x and selected_y:
        fig = px.scatter(df, x=selected_x, y=selected_y, size=selected_y, title='Bubble Chart', color=selected_x, template="plotly_dark")
    elif button_id == 'predict-btn' and 'Grades' in df.columns and selected_x:
        fig = px.scatter(df, x=selected_x, y='Grades', title=f'Prediction: Grades vs {selected_x}', color='Predicted Grades', template="plotly_dark")

    # Set consistent background styling
    fig.update_layout(
        plot_bgcolor='#34495E',
        paper_bgcolor='#2C3E50',
        font_color='white'
    )

    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
